In [ ]:
#https://keras.io/examples/generative/wgan_gp/
#https://keras.io/examples/generative/wgan-graphs/

In [ ]:
import os,shutil,random
from sklearn.preprocessing import MinMaxScaler
import joblib
from IPython.display import clear_output
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
from sklearn.model_selection import train_test_split
import datetime as dt
import numpy as np
from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

In [ ]:
#================== initialization ==================
#best LR = 1e-4,EPOCHS = 10,"discriminator_extra_steps":1
#best LR = 1e-4,EPOCHS = 15,"discriminator_extra_steps":1

currentTM=dt.datetime.now().strftime("%Y-%m-%dT%H%M%S")
PROJECT = "wgangpModel"
LATENT_DIM = 16
LR = 1e-4
EPOCHS = 5
BATCH_SIZE = 32

PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','TOTAL_VOLUME','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
MODEL_PATH = "./models/"
H5_PATH = './data/OptTrainData/'
STACK_DATA_SHAPE = np.empty((0,) + (16, len( SCALER_COL)  ) ) 

WANDB_LOG = True
RESUME = False
SUMMARY = True
log_dir = f"/app/logs/{PROJECT}/"+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
Scaler = joblib.load(SCALER_PATH )

#DISPLAY = ['map','summary',None]
DISPLAY = 'summary'


In [ ]:
##['UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','TOTAL_VOLUME','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
select_x = [i for i,c in  enumerate(SCALER_COL) if c in ['DTE','INTRINSIC_VALUE','C_VEGA','P_VEGA'] ]
select_y = [i for i,c in enumerate(SCALER_COL) if c in ['C_BID','C_ASK',  'P_BID',	'P_ASK'] ]

In [ ]:
notes = f"""
test Run use_bias set false , no tranfrom
"""
import wandb
from wandb.integration.keras import WandbCallback
CONFIG = {
          "learning_rate": LR,
          "epochs": EPOCHS,
          "batch_size": BATCH_SIZE,
          "architecture": "wgangp",
          "dataset": "OptionsChaine",
          "generator_dense_units":[128,64,32],
          "generator_dropout_rate":0.2,
          "discriminator_dense_units":[32,64,128],
          "discriminator_dropout_rate":0.2,
          "use_bias":False,
          "use_dropout":True,
          "use_bn":True,
          "transform":True,
          "discriminator_extra_steps":1,
          "x_col":select_x,
          "y_col":select_y,
           }

if WANDB_LOG :
    wandb.login()
    run = wandb.init(project=PROJECT, 
                     name=currentTM, 
                     config=CONFIG,
                     notes=notes
                    )

In [ ]:
#=====================================================================

In [ ]:
from src.wgangp.model import OptionChainGenerator
from src.wgangp.layer import generator, discriminator

model = OptionChainGenerator(
    discriminator = discriminator(
            input_shape= (16,len(select_y) ), 
            dense_units = CONFIG["generator_dense_units"], 
            dropout_rate= CONFIG["generator_dropout_rate"],
            use_bias=CONFIG["use_bias"],
            use_dropout=CONFIG["use_dropout"],
            use_bn=CONFIG["use_bn"]
           ), 
    generator = generator(
            input_dim = (16,len(select_y) ),
            output_dim = (16,len(select_x) ) ,
            dense_units = CONFIG["discriminator_dense_units"],
            dropout_rate= CONFIG["discriminator_dropout_rate"],
            use_bias=CONFIG["use_bias"],
            use_dropout=CONFIG["use_dropout"],
            use_bn=CONFIG["use_bn"]
           ),
    discriminator_extra_steps = CONFIG["discriminator_extra_steps"],
    output_col=[SCALER_COL[i] for i in select_y] ,
    scaler = Scaler
)


model.compile(
    d_optimizer = tf.optimizers.Adam(
    learning_rate=LR, beta_1=0.5, beta_2=0.9
    ),
    g_optimizer = tf.optimizers.Adam(
    learning_rate=LR, beta_1=0.5, beta_2=0.9
    )
)

In [ ]:
################## show model ######################
if DISPLAY == 'map' :
    from tensorflow.keras.utils import model_to_dot
    from IPython.display import SVG, display
    
    def display_model(model, width=1024, height=512):
        dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
        svg_data = dot.create(prog='dot', format='svg').decode("utf-8")
        svg_html = f'<div style="width:{width}px;height:{height}px;">{svg_data}</div>'
        display(HTML(svg_html))

In [ ]:
## Example usage:
## Display the generator model with reduced size
if DISPLAY == 'map' :
    display_model(model.generator, width, height=512)
if DISPLAY == 'summary' :
    model.generator.summary()

In [ ]:
if DISPLAY == 'map' :
    display_model(model.discriminator, width=2500, height=512)
if DISPLAY == 'summary' :
    model.discriminator.summary()

In [ ]:
#================== loadmodel ====================

In [ ]:
from tensorflow.keras.models import load_model
model_path = MODEL_PATH+f'{PROJECT}'
if not RESUME :
    if os.path.exists(model_path) :
        shutil.rmtree(model_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.generator.save(model_path+f'/'+f'generator.keras') 
    model.discriminator.save(model_path+f'/'+f'discriminator.keras') 
else:
    model.generator = load_model(model_path+'/'+f'generator.keras') 
    model.discriminator = load_model(model_path+'/'+f'discriminator.keras') 

In [ ]:
# ================== train model ==================
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
#SET MODEL VAR
STACK_DATA = STACK_DATA_SHAPE 
#INIT MODEL VAR
STOP_MODEL = False

#set PartitionDate[:] for limit range
for partdate in PartitionDate[:] :
    clear_output(wait=False)
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    ###transform
    if CONFIG['transform'] :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)
        #! เช็ตปัญหาการ แปลง shape
        #inv.tran data ไม่ได้ค่าเดิม
    DATA = np.vstack((DATA ,STACK_DATA))

    if len(DATA) < 64 :
        #stack data
        STACK_DATA = np.vstack((STACK_DATA ,DATA))
    else: 
        STACK_DATA = np.empty((0,) + data_shape[1:] )
        X = DATA[:, :, select_x]  # เลือกข้อมูล select_x สำหรับ X
        Y = DATA[:, :, select_y]  # เลือกข้อมูล select_y เสำหรับ Y
        x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
        random.shuffle(PartitionDate)
        tf.keras.backend.clear_session() 
        history = model.fit(x_train , y_train, epochs=CONFIG['epochs'], batch_size=BATCH_SIZE, validation_data=(x_val, y_val) ,callbacks=[tensorboard_callback])
        if  np.isnan(  np.average( history.history['generator_loss'] )  ) or np.isnan(  np.average( history.history['discriminator_loss'] )  ):
            STOP_MODEL = True 
    
        if WANDB_LOG :
            LogKeys = history.history.keys()
            LogVal={}
            for k in LogKeys:  
                LogVal[k] = np.average(  history.history[k] )
            wandb.log(LogVal, commit=True)
    if STOP_MODEL :
        break
    
            
    model.generator.save(model_path+f'/'+f'generator.keras') 
    model.discriminator.save(model_path+f'/'+f'discriminator.keras') 
if WANDB_LOG : wandb.finish()

In [ ]:
`====================================================

In [ ]:
x_train.shape

In [ ]:
y_train[8]

In [ ]:
ชช-=-----

In [ ]:
date_num = 1
partd = '2011-12'
with h5py.File(H5_PATH+partd+".h5", 'r') as f:
    d = f[partd][:]
#show sample data
d=Scaler.inverse_transform(
        d[15]
    )
d=pd.DataFrame(  
    d
,columns=SCALER_COL)
d

In [ ]:
#show zero data
d = Scaler.transform(
        np.zeros((16, 14))
    )
d=pd.DataFrame(  
    d
,columns=SCALER_COL)
d[:]

In [ ]:
======================== predict =========================

In [ ]:
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    print(f"CONFIG['transform'] : {CONFIG['transform']}")
    if CONFIG['transform'] or True :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)

In [ ]:
col_x = [SCALER_COL[i] for i in select_x]
col_y = [SCALER_COL[i] for i in select_y]

In [ ]:
X = DATA[:, :, select_x][:]
Y_real = DATA[:, :, select_y][:]
#===========
# X = x_train
# Y_real = y_train
# #===========
dfX = pd.DataFrame(
    X[:1].reshape(16, len(select_x)), 
    columns=col_x)
#print(dfX)

In [ ]:
genVal = model.generator(X) 

In [ ]:
dfY = pd.DataFrame(
    genVal.numpy()[:1].reshape(16, len(select_y)), 
    columns=col_y)
#print(dfY)

In [ ]:
SUMDF = pd.concat([dfX, dfY],axis=1)

In [ ]:
resultDF = pd.DataFrame([])
rm_col = []
for i in SCALER_COL:
    if i in SUMDF.columns:
         resultDF[i] = SUMDF[i]
    else:
        rm_col += [i]
        resultDF[i] = [1e-8]*16


In [ ]:
if CONFIG['transform'] :
    decode_transformed=Scaler.inverse_transform(
        resultDF
    )
    decode_transformed=pd.DataFrame(  
        decode_transformed
    ,columns=SCALER_COL).drop(columns=rm_col)
    decode_transformed.loc[decode_transformed['DTE'] < 1e-8] = 0
    decode_transformed[col_y] = decode_transformed[col_y].round(2)
decode_transformed

In [ ]:
========================

In [ ]:
tdVal = tf.constant( resultDF[['C_BID','C_ASK']].values )

In [ ]:
mask = tf.greater(tdVal[:,0], -0.502343)

In [ ]:
filtered_tensor = tf.boolean_mask(tdVal[:,0], mask)
filtered_tensor

In [ ]:
filtered_tensor_roll = tf.roll(filtered_tensor, shift=-1, axis=0)
filtered_tensor_roll

In [ ]:
difference = tf.maximum(
                filtered_tensor_roll[:-1] - filtered_tensor[:-1], 0.0
            )
difference

In [ ]:
======================= real data ==============================

In [ ]:
if CONFIG['transform'] :
    realdecode_transformed=Scaler.inverse_transform(
        DATA[0]
    )
    realdecode_transformed=pd.DataFrame(  
        realdecode_transformed
    ,columns=SCALER_COL).drop(columns=rm_col)
    realdecode_transformed.loc[realdecode_transformed['DTE'] <= 1e-8] = 0
    

In [ ]:
realdecode_transformed

In [ ]:
9.999994e-09

In [ ]:
======================= _compute_loss ==============================

In [ ]:
#generated_data = [c_bid, c_ask, c_volume, p_bid, p_ask, p_volume]
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]
generated_data = decode_data[3:]
z_mean    = z_mean
z_log_var = log_var
Y_real    = DATA[:, :, 3:][:1]

In [ ]:
z_mean

In [ ]:
for  col,genData in zip(colList,generated_data):
    print( colList.index(col),col )

In [ ]:
subtract_genData = genData - tf.cast(tf.expand_dims(Y_real[:, :, colList.index(col)], axis=-1)
        , tf.float32) 

In [ ]:
reconstruction_values_total = []
reconstruction_values_total.append( tf.reduce_mean( tf.square(subtract_genData)   ) )

In [ ]:
log_var = tf.clip_by_value(log_var, -1.0, 1.0)
kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(z_mean) - tf.exp(log_var), axis=-1)

In [ ]:
tf.reduce_mean(reconstruction_values_total + kl_loss)

In [ ]:
========== kiras vae origi

In [ ]:
Y_real[0][0]

In [ ]:
 tf.concat(decode_data, axis=-1).numpy()[0][0]

In [ ]:
Y_real[0] -  tf.concat(decode_data, axis=-1).numpy()

In [ ]:
features_loss = tf.reduce_mean(
    tf.reduce_sum(
        tf.keras.losses.mean_squared_error(Y_real, tf.concat(decode_data, axis=-1)),
        axis=(1),
    )
)

In [ ]:
features_loss

In [ ]:
# features_loss = tf.reduce_mean(
#     tf.reduce_sum(
#         keras.losses.categorical_crossentropy(features_real, features_gen),
#         axis=(1),
#     )
# )
# kl_loss = -0.5 * tf.reduce_sum(
#     1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), 1
# )
# kl_loss = tf.reduce_mean(kl_loss)

# property_loss = tf.reduce_mean(
#     keras.losses.binary_crossentropy(qed_true, qed_pred)
# )

# graph_loss = self._gradient_penalty(graph_real, graph_generated)

# return kl_loss + property_loss + graph_loss + adjacency_loss + features_loss

In [ ]:
======================= inverse_transform ========================

In [ ]:
#add 0
decode_data = [tf.zeros([1, 32, 1])]*3 + decode_data

In [ ]:
invert_decode = Scaler.inverse_transform(
    np.array([d.numpy().reshape(-1) for d in decode_data]).transpose()
    ) 

In [ ]:
pd.DataFrame(
    invert_decode[:,3:], 
    columns=SCALER_COL[3:])

In [ ]:
=====================================================================

In [ ]:
DATA.reshape(-1,data_shape[-1])

In [ ]:
data_shape

In [ ]:
import numpy as np
matrix = np.array([
[1,2,3,],
[4,5,6],
[7,8,9]
])

In [ ]:
matrix = tf.convert_to_tensor(matrix)
    
# Shift matrix to compare each row with the next row
matrix_next = tf.roll(matrix, shift=-1, axis=0)

# Ignore the last row for comparison as it rolls over to the first row
matrix = matrix[:-1]
matrix_next = matrix_next[:-1]

# Compute the difference between each row and the next row
difference = matrix_next - matrix

In [ ]:
matrix_next

In [ ]:
matrix

In [ ]:
difference

In [ ]:
import json


In [ ]:
json.dumps( """ WHERE 
Count_Date < LEFT( DATEADD( month,2,CAST('$timestamp' AS date) ),7)+'-01')
AND Count_Date > LEFT( DATEADD( year,-2,CAST('$timestamp' AS date) ),4)+'-01-01' 
""")